# Simple Expreiment: predict winner from champion lineup, gold & exp difference at 15 minute

In [687]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

## Dataset

Source: https://oracleselixir.com/tools/downloads

In [688]:
df = pd.read_csv('../data/2021_LoL_esports_match_data.csv', sep=',')

df_complete = df[df['datacompleteness'] == 'complete']
df_complete.reset_index(drop = True, inplace = True)
df = df_complete[df_complete['position'] == 'team']
df = df[df['side'] == 'Blue']
df.reset_index(drop = True, inplace = True)

print('# of matches: {}'.format(len(df)))

# of matches: 7268


## Dataset preprocessing

Add Winner for each match by merging two original columns:

In [689]:
df['winner'] = np.where(df['result']==1, 'blue', 'red')

df[['winner', 'result']].head()

,winner,result
0,blue,1
1,red,0
2,red,0
3,red,0
4,red,0



For this experiment, we only reserve golddiff at 15 min:

In [690]:
print(df['golddiffat15'].head())
print(df['xpdiffat15'].head())

0    5018.0
1     573.0
2    -579.0
3     951.0
4    2145.0
Name: golddiffat15, dtype: float64
0    4255.0
1   -1879.0
2   -1643.0
3    -107.0
4    -420.0
Name: xpdiffat15, dtype: float64


Get champion lineup stats:

In [691]:
champion_columns = [
       'blueTopChamp', 'blueJungleChamp', 'blueMiddleChamp', 'blueADCChamp', 'blueSupportChamp',
       'redTopChamp', 'redJungleChamp', 'redMiddleChamp', 'redADCChamp','redSupportChamp'
]
df[champion_columns] = np.NAN
# columns = champion_columns + ['golddiff', 'winner']
# df = df[columns]
champion_map_blue = {'top':'blueTopChamp', 'jng':'blueJungleChamp', 'mid':'blueMiddleChamp', 'bot':'blueADCChamp', 'sup':'blueSupportChamp'}
champion_map_red = {'top':'redTopChamp', 'jng':'redJungleChamp', 'mid':'redMiddleChamp', 'bot':'redADCChamp', 'sup':'redSupportChamp'}

for i in range(7268) :
    for j in range(5) :
        position = df_complete['position'][i * 12 + j]
        df[champion_map_blue[position]][i] = df_complete['champion'][i * 12 + j]
        position = df_complete['position'][i * 12 + 5 + j ]
        df[champion_map_red[position]][i] = df_complete['champion'][i * 12 + 5 + j]

columns = champion_columns + ['golddiffat15', 'xpdiffat15', 'winner']
df = df[columns]

df.head()

,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp,golddiffat15,xpdiffat15,winner
0,Mordekaiser,Graves,Zoe,Miss Fortune,Galio,Ornn,Kindred,Orianna,Ezreal,Leona,5018.0,4255.0,blue
1,Gragas,Graves,Rumble,Kai'Sa,Alistar,Ornn,Olaf,Syndra,Miss Fortune,Galio,573.0,-1879.0,red
2,Gragas,Graves,Zoe,Yasuo,Alistar,Rumble,Nidalee,Yone,Miss Fortune,Galio,-579.0,-1643.0,red
3,Karma,Pantheon,Syndra,Samira,Leona,Aatrox,Nidalee,Viktor,Kai'Sa,Galio,951.0,-107.0,red
4,Jax,Hecarim,Orianna,Samira,Alistar,Camille,Lillia,Viktor,Kai'Sa,Maokai,2145.0,-420.0,red


Encode labels:

In [692]:
champion_label_encoder = LabelEncoder()
champions = set()
for champ in champion_columns :
    champions |= set(pd.unique(df[champ]))
print(champions)
champions = np.array(list(champions))

champion_label_encoder.fit(champions)
for champ in champion_columns :
    df[champ] = champion_label_encoder.transform(df[champ])

label_encoder = LabelEncoder()
df['winner'] = label_encoder.fit_transform(df['winner'])

df.head()

{'Thresh', 'Mordekaiser', 'Gangplank', 'Soraka', 'Maokai', 'Dr. Mundo', 'Quinn', 'Swain', 'Ornn', 'Jayce', 'Kayle', 'Aphelios', 'Rengar', 'Lissandra', 'Lee Sin', 'Pyke', 'Talon', 'Amumu', 'Evelynn', 'Lucian', 'Elise', 'Nasus', 'Skarner', 'Zyra', 'Sivir', 'Jarvan IV', 'Ahri', 'Sion', 'Zed', "Rek'Sai", 'Wukong', 'Samira', 'Garen', 'Nidalee', 'Ezreal', 'Olaf', 'Trundle', 'Warwick', 'Tahm Kench', 'Sylas', 'Rumble', 'Nunu & Willump', "Vel'Koz", 'Veigar', 'Tryndamere', 'Orianna', 'Xayah', 'Braum', 'Yorick', 'Kled', 'Morgana', 'Yuumi', 'Azir', 'Brand', 'Viktor', 'Heimerdinger', 'Riven', 'Taliyah', 'Nautilus', 'Anivia', 'Tristana', 'Gnar', 'Katarina', 'Yasuo', 'Shaco', 'Twitch', 'Urgot', 'Vex', 'Ivern', 'Hecarim', "Kha'Zix", 'Xin Zhao', 'Corki', 'Kassadin', 'Shyvana', 'Blitzcrank', 'Draven', 'Sett', 'Karma', 'Viego', 'Ashe', 'Gragas', "Kai'Sa", 'Vi', 'Janna', 'Volibear', 'Malzahar', 'Akshan', 'Alistar', 'Taric', 'Nocturne', 'Zoe', 'Lux', 'Kindred', 'Fiora', 'Aurelion Sol', 'Diana', 'Varus', 'N

,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp,golddiffat15,xpdiffat15,winner
0,75,37,153,74,32,86,60,85,28,65,5018.0,4255.0,0
1,36,37,99,50,4,86,84,116,74,32,573.0,-1879.0,1
2,36,37,153,145,4,99,81,146,74,32,-579.0,-1643.0,1
3,52,87,116,101,65,0,81,137,50,32,951.0,-107.0,1
4,46,39,85,101,4,17,66,137,50,73,2145.0,-420.0,1


Normalize Data:

In [693]:
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
df = pd.DataFrame(df, columns = columns)

# for champ in champion_columns :
#     df[champ] = label_encoder.fit_transform(df[champ])
df['winner'] = label_encoder.fit_transform(df['winner'])

df.head()

,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp,golddiffat15,xpdiffat15,winner
0,0.357028,-1.246428,1.448543,0.005258,-0.911696,0.649750,-0.564821,-0.062106,-1.022908,-0.053170,1.496628,1.875184,0
1,-0.618736,-1.246428,0.230645,-0.546604,-1.594797,0.649750,0.101829,0.618518,0.019068,-0.869177,0.089622,-0.848318,1
2,-0.618736,-1.246428,1.448543,1.637851,-1.594797,0.971295,0.018498,1.277185,0.019068,-0.869177,-0.275028,-0.743534,1
3,-0.218422,0.128487,0.614057,0.626103,-0.106612,-1.477393,0.018498,1.079585,-0.524572,-0.869177,0.209273,-0.061548,1
4,-0.368540,-1.191432,-0.085107,0.626103,-1.594797,-1.056911,-0.398159,1.079585,-0.524572,0.144650,0.587218,-0.200521,1


Finally, we get our x and y data for training.

In [694]:
x = df.drop(['winner'], axis = 1)
y = df['winner']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# x_train = np.array(x_train).reshape(-1, 1)
# x_test = np.array(x_test).reshape(-1, 1)

## Train & Test

Train Logistic Regression model and test its accuracy.

In [695]:
model_LR = LogisticRegression()
model_LR.fit(x_train, y_train)
y_pred = model_LR.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.80       386
           1       0.77      0.76      0.76       341

    accuracy                           0.78       727
   macro avg       0.78      0.78      0.78       727
weighted avg       0.78      0.78      0.78       727



Test/predict single one match:

In [696]:
golddiffat15 = -795
xpdiffat15 = -1087

champion_lineup_blue = ['Graves', 'Trundle', 'Syndra', 'Ziggs', 'Leona'] # DK
champion_lineup_red = ['Kennen', 'Xin Zhao', 'Zoe', 'Aphelios', 'Rakan'] # EDG

# champion_lineup_blue = ['Gwen', 'Lee Sin','LeBlanc', 'Aphelios', 'Lulu']
# champion_lineup_red = ['Graves', 'Jarvan IV', 'Twisted Fate','Lucian','Nami']

# champion_lineup_blue = ['Gnar', 'Rengar', 'Ahri', 'Caitlyn', 'Leona']
# champion_lineup_red = ['Irelia', 'Jarvan IV', 'Azir', 'Corki', 'Annie']

# champion_lineup_blue = ['Jayce', 'Lee Sin', 'Twisted Fate', 'Miss Fortune', 'Leona']
# champion_lineup_red = ['Kennen', 'Jarvan IV', 'Azir', 'Vayne', 'Lulu']

x = [np.concatenate([champion_label_encoder.transform(champion_lineup_blue), champion_label_encoder.transform(champion_lineup_red) , [golddiffat15, xpdiffat15, 0]])]
x = scaler.transform(x)[:, :-1]
y_pred = model_LR.predict_proba(x)

print(model_LR.predict(x))
print("blue wins: {:.1f}% red wins: {:.1f}%".format(y_pred[0][0] * 100, y_pred[0][1] * 100))

[1]
blue wins: 38.9% red wins: 61.1%
